In [1]:
# To be run after Adult-Data-Prep
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import balanced_accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
#from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_Adult
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Adult dataset
Adult_df = pd.read_csv('./input/adult-cleaned.csv')
# The AIF demo drops the following columns - we'll try the same
Adult_df.drop(["Fnlwgt"],axis=1,inplace=True)
#Adult_df.drop(["Fnlwgt", "NativeCountry", "Relationship", "MaritalStatus"],axis=1,inplace=True)

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'Gender'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'Income'
fav_label           = 1 # Income over £50K
unfav_label         = 0 # Income under £50K
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create the traoning and test splits
X = Adult_df.drop(lab,axis=1)
y = Adult_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
Adult_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Adult_train_bld, Adult_test_bld = Adult_bld.split([0.8], shuffle=True)

In [7]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
Adult_train_bld.features = min_max_scaler.fit_transform(Adult_train_bld.features)
Adult_test_bld.features  = min_max_scaler.fit_transform(Adult_test_bld.features)

In [8]:
# Convert the train and test datasets to dataframes
Adult_train_df, d = Adult_train_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df,  d = Adult_test_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [9]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Adult_train_df.drop(lab,axis=1)
y_train = Adult_train_df[lab]
X_test  = Adult_test_df.drop(lab,axis=1)
y_test  = Adult_test_df[lab]
BiasedLogModel = LogisticRegression(random_state=101)
BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
BiasedLogModel.fit(X_train, y_train) 
BiasedRfcModel.fit(X_train, y_train)
BiasedLogPredictions = BiasedLogModel.predict(X_test)
BiasedRfcPredictions = BiasedRfcModel.predict(X_test)
print(f"Biased Logistic regression validation accuracy: {BiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {BiasedRfcModel.score(X_test, y_test)}")
print('')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, BiasedLogPredictions)}")
print(f"Biased Random forest balanced accuracy        : {balanced_accuracy_score(y_test, BiasedRfcPredictions)}")

Biased Logistic regression validation accuracy: 0.8244446719213839
Biased Random Forest       validation accuracy: 0.8384686252431159

Biased Logistic regression balanced accuracy  : 0.6875551809078394
Biased Random forest balanced accuracy        : 0.6779616210211661


In [10]:
print('Training Before - Income value counts:')
print(Adult_train_df.Income.value_counts())
print('Training Before - Gender value counts:')
print(Adult_train_df.Gender.value_counts())

Training Before - Income value counts:
0.0    29710
1.0     9363
Name: Income, dtype: int64
Training Before - Gender value counts:
1.0    26142
0.0    12931
Name: Gender, dtype: int64


In [11]:
print('Test Before - Income value counts:')
print(Adult_test_df.Income.value_counts())
print('Test Before - Gender value counts:')
print(Adult_test_df.Gender.value_counts())

Test Before - Income value counts:
0.0    7445
1.0    2324
Name: Income, dtype: int64
Test Before - Gender value counts:
1.0    6508
0.0    3261
Name: Gender, dtype: int64


In [12]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(Adult_train_bld, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_test_bld = BinaryLabelDatasetMetric(Adult_test_bld, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

In [13]:
display(Markdown("#### Orig training dataset"))
print('Number of instances           :', metric_train_bld.num_instances())
print('Base Rate                     :', metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
display(Markdown("#### Orig test dataset"))
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

#### Orig training dataset

Number of instances           : 39073.0
Base Rate                     : 0.23962838788933535
Consistency                   : [0.8318532]
Disparate Impact              : 0.3578237554537522
Mean Difference               : -0.19541397082722825
Statistical Parity Difference : -0.19541397082722825


#### Orig test dataset

Number of instances           : 9769.0
Base Rate                     : 0.23789538335551233
Consistency                   : [0.82456751]
Disparate Impact              : 0.36701485922563437
Mean Difference               : -0.19092652602029497
Statistical Parity Difference : -0.19092652602029497


In [14]:
#Test for equality of odds. 
Test_Log_bld = Adult_test_bld.copy(deepcopy=True)
Test_Rfc_bld = Adult_test_bld.copy(deepcopy=True)
TestLogPredictions = BiasedLogModel.predict(X_test)
TestRfcPredictions = BiasedRfcModel.predict(X_test)
Test_Log_bld.labels= TestLogPredictions
Test_Rfc_bld.labels= TestRfcPredictions

c_Log_metric = ClassificationMetric(Adult_test_bld, Test_Log_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
c_Rfc_metric = ClassificationMetric(Adult_test_bld, Test_Rfc_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)

In [15]:
print('A value of 0 means that equality of odds has been met')
print('Log average_abs_odds_difference:', c_Log_metric.average_abs_odds_difference())
print('Log average_odds_difference:    ', c_Log_metric.average_odds_difference())
print(' ')
print('Rfc average_abs_odds_difference:', c_Rfc_metric.average_abs_odds_difference())
print('Rfc average_odds_difference:    ', c_Rfc_metric.average_odds_difference())

A value of 0 means that equality of odds has been met
Log average_abs_odds_difference: 0.1788771774875064
Log average_odds_difference:     -0.1788771774875064
 
Rfc average_abs_odds_difference: 0.10458697728436953
Rfc average_odds_difference:     -0.10458697728436953


In [16]:
# Test for Equality of odds
Test_bld = BinaryLabelDataset(df=pd.concat((X_test, y_test),axis=1),
                              label_names=[lab], protected_attribute_names=[protected_attr],
                              favorable_label=fav_label, unfavorable_label=unfav_label)
Test_Log_bld = Test_bld.copy(deepcopy=True)
Test_Rfc_bld = Test_bld.copy(deepcopy=True)
TestLogPredictions = BiasedLogModel.predict(X_test)
TestRfcPredictions = BiasedRfcModel.predict(X_test)
Test_Log_bld.labels= TestLogPredictions
Test_Rfc_bld.labels= TestRfcPredictions#

c_Log_metric = ClassificationMetric(Test_bld, Test_Log_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
c_Rfc_metric = ClassificationMetric(Test_bld, Test_Rfc_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)

In [17]:
print('A value of 0 means that equality of odds has been met')
print('Log average_abs_odds_difference:', c_Log_metric.average_abs_odds_difference())
print('Log average_odds_difference:    ', c_Log_metric.average_odds_difference())
print(' ')
print('Rfc average_abs_odds_difference:', c_Rfc_metric.average_abs_odds_difference())
print('Rfc average_odds_difference:    ', c_Rfc_metric.average_odds_difference())

A value of 0 means that equality of odds has been met
Log average_abs_odds_difference: 0.1788771774875064
Log average_odds_difference:     -0.1788771774875064
 
Rfc average_abs_odds_difference: 0.10458697728436953
Rfc average_odds_difference:     -0.10458697728436953


In [18]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [19]:
biased_model.fit(Adult_train_bld)

W0910 19:14:44.673133  2100 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0910 19:14:44.675139  2100 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0910 19:14:44.694133  2100 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0910 19:14:46.732129  2100 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.725036
epoch 0; iter: 200; batch classifier loss: 0.442806
epoch 1; iter: 0; batch classifier loss: 0.412049
epoch 1; iter: 200; batch classifier loss: 0.410577
epoch 2; iter: 0; batch classifier loss: 0.338738
epoch 2; iter: 200; batch classifier loss: 0.347446
epoch 3; iter: 0; batch classifier loss: 0.339487
epoch 3; iter: 200; batch classifier loss: 0.285346
epoch 4; iter: 0; batch classifier loss: 0.341591
epoch 4; iter: 200; batch classifier loss: 0.331817
epoch 5; iter: 0; batch classifier loss: 0.351455
epoch 5; iter: 200; batch classifier loss: 0.321800
epoch 6; iter: 0; batch classifier loss: 0.288243
epoch 6; iter: 200; batch classifier loss: 0.376674
epoch 7; iter: 0; batch classifier loss: 0.288993
epoch 7; iter: 200; batch classifier loss: 0.340328
epoch 8; iter: 0; batch classifier loss: 0.338564
epoch 8; iter: 200; batch classifier loss: 0.279655
epoch 9; iter: 0; batch classifier loss: 0.333028
epoch 9; iter: 200; batch classi

In [20]:
# Apply the plain model to test data
Adult_biased_train = biased_model.predict(Adult_train_bld)
Adult_biased_test  = biased_model.predict(Adult_test_bld)
# And convert them to dataframes
Adult_train_df, d = Adult_biased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df, d  = Adult_biased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [21]:
# Test whether it is possible to predict the Sensitive Variable from the biased dataset
X_se_train = Adult_train_df.drop(protected_attr,axis=1)
y_se_train = Adult_train_df[protected_attr]
X_se_test  = Adult_test_df.drop(protected_attr,axis=1)
y_se_test  = Adult_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7525847067253557
Biased Random Forest       validation accuracy: 0.7956802129184154

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.6892301259394
Biased Random Forest       balanced accuracy  : 0.7501854392122205


In [22]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_biased_train = BinaryLabelDatasetMetric(Adult_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_biased_test = BinaryLabelDatasetMetric(Adult_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_biased_train.num_instances())
print('Base Rate                     :', metric_biased_train.base_rate())
print('Consistency                   :', metric_biased_train.consistency())
print('Disparate Impact              :', metric_biased_train.disparate_impact())
print('Mean Difference               :', metric_biased_train.mean_difference())
print('Statistical Parity Difference :', metric_biased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_biased_test.num_instances())
print('Base Rate                     :', metric_biased_test.base_rate())
print('Consistency                   :', metric_biased_test.consistency())
print('Disparate Impact              :', metric_biased_test.disparate_impact())
print('Mean Difference               :', metric_biased_test.mean_difference())
print('Statistical Parity Difference :', metric_biased_test.statistical_parity_difference()) 

#### Plain model - without debiasing - dataset metrics

#### Biased training dataset

Number of instances           : 39073.0
Base Rate                     : 0.19704143526220153
Consistency                   : [0.96067361]
Disparate Impact              : 0.2852547840972396
Mean Difference               : -0.18446889953698742
Statistical Parity Difference : -0.18446889953698742


#### Biased test dataset

Number of instances           : 9769.0
Base Rate                     : 0.19766608660047089
Consistency                   : [0.94468216]
Disparate Impact              : 0.27652777536697276
Mean Difference               : -0.18853855147166781
Statistical Parity Difference : -0.18853855147166781


In [23]:
# Create the Classification metric for train and test
classified_metric_biased_test = ClassificationMetric(Adult_train_bld, Adult_biased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_biased_train = ClassificationMetric(Adult_test_bld, Adult_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [24]:
display(Markdown("#### Plain model - without debiasing - classification metrics"))

display(Markdown("#### Biased training dataset - CLassification Metrics"))
TPR_train = classified_metric_biased_train.true_positive_rate()
TNR_train = classified_metric_biased_train.true_negative_rate()
bal_acc_biased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_biased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_train)
print('Disparate impact                =', classified_metric_biased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_train.theil_index())

display(Markdown("#### Biased test dataset - CLassification Metrics"))
TPR_test = classified_metric_biased_test.true_positive_rate()
TNR_test = classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_biased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_test)
print('Disparate impact                =', classified_metric_biased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - classification metrics

#### Biased training dataset - CLassification Metrics

Classification accuracy         = 0.8535162247927116
Balanced classification accuracy= 0.7689309670804488
Disparate impact                = 0.27652777536697276
Equal opportunity difference    = -0.15851140842427008
Average odds difference         = -0.11827622695406441
Theil_index                     = 0.11779794438022112


#### Biased test dataset - CLassification Metrics

Classification accuracy         = 0.8538632815499194
Balanced classification accuracy= 0.7690622023790586
Disparate impact                = 0.2852547840972396
Equal opportunity difference    = -0.10077361650762806
Average odds difference         = -0.0904799397691296
Theil_index                     = 0.11848825605542657


In [25]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [26]:
unbiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [27]:
unbiased_model.fit(Adult_train_bld)

epoch 0; iter: 0; batch classifier loss: 0.688806; batch adversarial loss: 0.708610
epoch 0; iter: 200; batch classifier loss: 0.549827; batch adversarial loss: 0.675114
epoch 1; iter: 0; batch classifier loss: 0.526568; batch adversarial loss: 0.644793
epoch 1; iter: 200; batch classifier loss: 0.454933; batch adversarial loss: 0.652382
epoch 2; iter: 0; batch classifier loss: 0.485578; batch adversarial loss: 0.667268
epoch 2; iter: 200; batch classifier loss: 0.387557; batch adversarial loss: 0.659195
epoch 3; iter: 0; batch classifier loss: 0.368546; batch adversarial loss: 0.633070
epoch 3; iter: 200; batch classifier loss: 0.370732; batch adversarial loss: 0.641306
epoch 4; iter: 0; batch classifier loss: 0.328418; batch adversarial loss: 0.672769
epoch 4; iter: 200; batch classifier loss: 0.288229; batch adversarial loss: 0.632936
epoch 5; iter: 0; batch classifier loss: 0.375741; batch adversarial loss: 0.585835
epoch 5; iter: 200; batch classifier loss: 0.323261; batch adversa

epoch 48; iter: 200; batch classifier loss: 0.319905; batch adversarial loss: 0.607919
epoch 49; iter: 0; batch classifier loss: 0.349859; batch adversarial loss: 0.629609
epoch 49; iter: 200; batch classifier loss: 0.372623; batch adversarial loss: 0.594221


In [28]:
Adult_unbiased_train = unbiased_model.predict(Adult_train_bld)
Adult_unbiased_test  = unbiased_model.predict(Adult_test_bld)
# And convert these to dataframes..
Adult_train_df, d = Adult_unbiased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df, d  = Adult_unbiased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [29]:
# Metrics for the dataset from plain model (after unbiasing)
display(Markdown("#### Plain model - after unbiasing - dataset metrics"))
metric_unbiased_train = BinaryLabelDatasetMetric(Adult_unbiased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_unbiased_test = BinaryLabelDatasetMetric(Adult_unbiased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_unbiased_train.num_instances())
print('Base Rate                     :', metric_unbiased_train.base_rate())
print('Consistency                   :', metric_unbiased_train.consistency())
print('Disparate Impact              :', metric_unbiased_train.disparate_impact())
print('Mean Difference               :', metric_unbiased_train.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_unbiased_test.num_instances())
print('Base Rate                     :', metric_unbiased_test.base_rate())
print('Consistency                   :', metric_unbiased_test.consistency())
print('Disparate Impact              :', metric_unbiased_test.disparate_impact())
print('Mean Difference               :', metric_unbiased_test.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_test.statistical_parity_difference()) 

#### Plain model - after unbiasing - dataset metrics

#### Biased training dataset

Number of instances           : 39073.0
Base Rate                     : 0.15596447674864997
Consistency                   : [0.95782766]
Disparate Impact              : 0.7506138828251397
Mean Difference               : -0.04239430436558331
Statistical Parity Difference : -0.04239430436558331


#### Biased test dataset

Number of instances           : 9769.0
Base Rate                     : 0.160405363906234
Consistency                   : [0.94296243]
Disparate Impact              : 0.7212980406355712
Mean Difference               : -0.04929101954954787
Statistical Parity Difference : -0.04929101954954787


In [30]:
# Create the Classification metric for train and test
classified_metric_unbiased_train = ClassificationMetric(Adult_train_bld, Adult_unbiased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_unbiased_test = ClassificationMetric(Adult_test_bld, Adult_unbiased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [31]:
display(Markdown("#### Plain model - after debiasing - classification metrics"))
display(Markdown("#### Unbiased training dataset - CLassification Metrics"))
TPR_train = classified_metric_unbiased_train.true_positive_rate()
TNR_train = classified_metric_unbiased_train.true_negative_rate()
bal_acc_unbiased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_unbiased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_train)
print('Disparate impact                =', classified_metric_unbiased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_train.theil_index())

display(Markdown("#### Unbiased test dataset - CLassification Metrics"))
TPR_test = classified_metric_unbiased_test.true_positive_rate()
TNR_test = classified_metric_unbiased_test.true_negative_rate()
bal_acc_unbiased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_unbiased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_test)
print('Disparate impact                =', classified_metric_unbiased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_test.theil_index())

#### Plain model - after debiasing - classification metrics

#### Unbiased training dataset - CLassification Metrics

Classification accuracy         = 0.8418601080029688
Balanced classification accuracy= 0.7232437234683395
Disparate impact                = 0.7506138828251397
Equal opportunity difference    = 0.25027132378149824
Average odds difference         = 0.1314215808525129
Theil_index                     = 0.14370818831644128


#### Unbiased test dataset - CLassification Metrics

Classification accuracy         = 0.8434844917596479
Balanced classification accuracy= 0.7281643411408274
Disparate impact                = 0.7212980406355712
Equal opportunity difference    = 0.1624950786220989
Average odds difference         = 0.09046703022565815
Theil_index                     = 0.14003453898232962


In [32]:
# Determine the after-transformation model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Adult_train_df.drop(lab,axis=1)
y_train = Adult_train_df[lab]
X_test  = Adult_test_df.drop(lab,axis=1)
y_test  = Adult_test_df[lab]

UnbiasedLogModel = LogisticRegression(random_state=101)
UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_train, y_train) 
UnbiasedRfcModel.fit(X_train, y_train) 
UnbiasedLog_pred = UnbiasedLogModel.predict(X_test)
UnbiasedRfc_pred = UnbiasedRfcModel.predict(X_test)

# Now get Logistic Regression and Random Forest Clasisfication performancce of unbiased data
print('Model Accuracy for Log Reg and RFC after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {UnbiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {UnbiasedRfcModel.score(X_test, y_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedRfc_pred)}")

Model Accuracy for Log Reg and RFC after bias transformation:
Biased Logistic regression validation accuracy: 0.9002968574060805
Biased Random Forest       validation accuracy: 0.9000921281605078

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.7379997983277072
Biased Random Forest       balanced accuracy  : 0.694255545248898


In [36]:
# Test for Equality of odds
Test_bld = BinaryLabelDataset(df=pd.concat((X_test, y_test),axis=1),
                              label_names=[lab], protected_attribute_names=[protected_attr],
                              favorable_label=fav_label, unfavorable_label=unfav_label)
Test_Log_bld = Test_bld.copy(deepcopy=True)
Test_Rfc_bld = Test_bld.copy(deepcopy=True)
TestLogPredictions = UnbiasedLogModel.predict(X_test)
TestRfcPredictions = UnbiasedRfcModel.predict(X_test)
Test_Log_bld.labels= TestLogPredictions
Test_Rfc_bld.labels= TestRfcPredictions#

c_Log_metric = ClassificationMetric(Test_bld, Test_Log_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
c_Rfc_metric = ClassificationMetric(Test_bld, Test_Rfc_bld, 
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)

In [37]:
print('A value of 0 means that equality of odds has been met')
print('Log average_abs_odds_difference:', c_Log_metric.average_abs_odds_difference())
print('Log average_odds_difference:    ', c_Log_metric.average_odds_difference())
print(' ')
print('Rfc average_abs_odds_difference:', c_Rfc_metric.average_abs_odds_difference())
print('Rfc average_odds_difference:    ', c_Rfc_metric.average_odds_difference())

A value of 0 means that equality of odds has been met
Log average_abs_odds_difference: 0.04989007365531851
Log average_odds_difference:     -0.04989007365531851
 
Rfc average_abs_odds_difference: 0.09027473904298801
Rfc average_odds_difference:     -0.09027473904298801


In [33]:
# Finally test whether it is possible to predict the Sensitive Variable from the transformed dataset
X_se_train = Adult_train_df.drop(protected_attr,axis=1)
y_se_train = Adult_train_df[protected_attr]
X_se_test  = Adult_test_df.drop(protected_attr,axis=1)
y_se_test  = Adult_test_df[protected_attr]

Se_unbiasedLogModel = LogisticRegression(random_state=101)
Se_unbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_unbiasedLogModel.fit(X_se_train, y_se_train) 
Se_unbiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_unbiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_unbiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_unbiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_unbiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7537107175760057
Biased Random Forest       validation accuracy: 0.7934281912171154

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.6840318438071737
Biased Random Forest       balanced accuracy  : 0.7488777052073008


In [34]:
print('Predicted Training After - Income value counts:')
print(Adult_train_df.Income.value_counts())
print('Predicted Training After - Gender value counts:')
print(Adult_train_df.Gender.value_counts())

Predicted Training After - Income value counts:
0.0    32979
1.0     6094
Name: Income, dtype: int64
Predicted Training After - Gender value counts:
1.0    26142
0.0    12931
Name: Gender, dtype: int64


In [35]:
print('Predicted Test After - Income value counts:')
print(Adult_test_df.Income.value_counts())
print('Predicted Test After - Gender value counts:')
print(Adult_test_df.Gender.value_counts())

Predicted Test After - Income value counts:
0.0    8202
1.0    1567
Name: Income, dtype: int64
Predicted Test After - Gender value counts:
1.0    6508
0.0    3261
Name: Gender, dtype: int64
